In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

In [2]:
OSM_PATH = "example.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

In [3]:
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [4]:
SCHEMA = schema.py

In [5]:
# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [6]:
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

In [7]:
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


In [8]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [9]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])



In [10]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    if element.tag == 'node':
        for i in node_attr_fields:
            node_attribs[i]=element.attrib[i]
        
        for second_tag in element.iter('tag'):
            if PROBLEMCHARS.match(second_tag.attrib['k']):
                break
            else:
                tag_elements = {}
                tag_elements['id'] = element.attrib['id']
                tag_elements['value'] = second_tag.attrib['v']
                if ':' in second_tag.attrib['k']:
                    k_value = second_tag.attrib['k'].split(':',1)
                    tag_elements['type'] = k_value[0]
                    tag_elements['key'] = k_value[1]
                else:
                    tag_elements['type'] = default_tag_type
                    tag_elements['key'] = second_tag.attrib['k']
                tags.append(tag_elements)
                
        return({'node': node_attribs, 'node_tags': tags})
    
    elif element.tag == 'way':
        
        for i in WAY_FIELDS:
            way_attribs[i]=element.attrib[i]
        
        for second_tag in element.iter('tag'):
            if PROBLEMCHARS.match(second_tag.attrib['k']):
                    break
            else:
                tag_elements = {}
                tag_elements['id'] = element.attrib['id']
                tag_elements['value'] = second_tag.attrib['v']
                if ':' in second_tag.attrib['k']:
                    k_value = second_tag.attrib['k'].split(':',1)
                    tag_elements['type'] = k_value[0]
                    tag_elements['key'] = k_value[1]
                else:
                    tag_elements['type'] = 'regular'
                    tag_elements['key'] = second_tag.attrib['k']
                tags.append(tag_elements)
        
        pos = 0
        for node in element.iter('nd'):
            waynd_dt = {}
            waynd_dt['id'] = element.attrib['id']
            waynd_dt['node_id'] = node.attrib['ref']
            waynd_dt['position'] = pos
            pos += 1
            way_nodes.append(waynd_dt)
            
        return({'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags})
            

In [11]:
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

SchemaError: schema definition for field 'schema.py' must be a dict

In [ ]:
file_in = "example.osm.xml"
way_attribs = {}
way_nodes = []
tags = []  # Handle secondary tags the same way for both node and way elements
for element in get_element(file_in, tags=('node', 'way')):
    if element.tag == 'way':
        
        for i in WAY_FIELDS:
            way_attribs[i]=element.attrib[i]
        
        for second_tag in element.iter('tag'):
            if PROBLEMCHARS.match(second_tag.attrib['k']):
                    break
            else:
                tag_elements = {}
                tag_elements['id'] = element.attrib['id']
                tag_elements['value'] = second_tag.attrib['v']
                if ':' in second_tag.attrib['k']:
                    k_value = second_tag.attrib['k'].split(':',1)
                    tag_elements['type'] = k_value[0]
                    tag_elements['key'] = k_value[1]
                else:
                    tag_elements['type'] = 'regular'
                    tag_elements['key'] = second_tag.attrib['k']
                tags.append(tag_elements)
        
        pos = 0
        for node in element.iter('nd'):
            waynd_dt = {}
            waynd_dt['id'] = element.attrib['id']
            waynd_dt['node_id'] = node.attrib['ref']
            waynd_dt['id'] = element.attrib['id']
            way_nodes.append(waynd_dt)
            
        #return({'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags})
        #print('way ', way_attribs)
         #   , 'way_nodes': way_nodes, 'way_tags': tags}